<a href="https://colab.research.google.com/github/smartgrids-aau/GreenCodesPython/blob/main/Exemple_6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Exercise 6.1

import numpy as np
import pandas as pd
from scipy import integrate
import sympy as sp
import warnings
warnings.filterwarnings('ignore')

# Modeling of PV system using Python
fileName = 'PV Modeling Book Data Source.xls'
sheetName = 'Source 7'

# Read Excel data
df = pd.read_excel(fileName, sheet_name=sheetName)
I_L = df.iloc[9:8769, 3].values     # Column 'L'
T   = df.iloc[9:8769, 1].values     # Column ' temp.'
S = df.iloc[9:8769, 0].values

NOCT = 43.6
Alpha = 0.068  # Temperature Coefficient
IPV_M = 6.89   # Current at MPP

# Initialize arrays
T_cell = np.zeros(len(S))
I_PV = np.zeros(len(S))

for k in range(len(S)):  # Number of hours
    T_cell[k] = (T[k] + (((NOCT - 20) / 800) * S[k]))  # Cell Temp
    I_PV[k] = ((S[k] / 1000)) + (Alpha * (T_cell[k] - 25))

for zzz in range(len(I_L)):
    if I_PV[zzz] < 0:
        I_PV[zzz] = 0
    else:
        I_PV[zzz] = I_PV[zzz]

k = 0
zzz = 0

# (Routine 2): Initialization
PV_eff = 0.14    # PV module efficiency
Wire_eff = 0.98  # Wire efficiency
INV_eff = 0.95   # inverter efficiency
V_sys = 230      # System voltage (V)
PSH = (np.mean(S) * 12) / 1000  # Peak sun shine hours
V_B = 12         # Battery voltage
DCharge_eff = 0.8  # Battery charging efficiency
DOD = 0.8        # Allowed Depth of charge (%)
Alpha = 0.068
PV_WP = 120

# (Routine 3): Intuitive method to find the initial N of PV modules needed
P_L = np.zeros(len(I_L))
for i in range(len(I_L)):
    P_L[i] = I_L[i] * 230

A_PV = 1.408 * 0.56

# Fix: Calculate total energy consumption correctly
E_L = np.sum(I_L * 230)  # Total energy consumption in Wh

# Check for valid values before proceeding
if np.isnan(E_L) or E_L <= 0:
    print("Error: Invalid energy consumption calculation")
    print(f"E_L = {E_L}")
    print(f"I_L stats: min={np.min(I_L)}, max={np.max(I_L)}, mean={np.mean(I_L)}")
    # Set a default value or exit
    E_L = 1000  # Default value for testing

if np.isnan(PSH) or PSH <= 0:
    print("Error: Invalid PSH calculation")
    print(f"PSH = {PSH}")
    print(f"S stats: min={np.min(S)}, max={np.max(S)}, mean={np.mean(S)}")
    PSH = 5  # Default value for testing

N_PV = np.ceil((E_L / (PV_eff * INV_eff * Wire_eff * PSH * 1000 * A_PV)))

# Check if N_PV is valid
if np.isnan(N_PV) or N_PV <= 0:
    print("Error: Invalid N_PV calculation")
    print(f"N_PV = {N_PV}")
    N_PV = 10  # Default value

N_PVmin = int(np.ceil(N_PV / 3))
N_PVmax = int(5 * N_PV)
nh = 2
C_battery = (E_L * nh) / (DOD * DCharge_eff)
C_batterymin = C_battery / 3
C_batterymax = 5 * C_battery

print(f"E_L = {E_L}")
print(f"PSH = {PSH}")
print(f"N_PV = {N_PV}")
print(f"N_PVmin = {N_PVmin}, N_PVmax = {N_PVmax}")
print(f"C_battery = {C_battery}")
print(f"C_batterymin = {C_batterymin}, C_batterymax = {C_batterymax}")

# (Routine 4): Find LLP at each Battery Capacity and PV module trial
SOC = []
I_Load = []
I_Charge = []
I_Discharge = []
I_Deficit = []
I_Damp = []
I_Battery = []
t = 1
Vmp = 17.4
NOCT = 43.6
K = 0.8
D = 1e-5
ns = 6  # Number of cells per battery
SOC1 = 1  # Max. battery SOC=100% of total capacity
SOC2 = SOC1
SOC3 = 0.2  # Min. battery SOC=20% of total capacity

# Initialize result matrices
C_batteryf = []
C_PVf = []
LLP_calculated = []

x = 0
for m in range(N_PVmin, N_PVmax + 1):  # Number of PV modules
    y = 0
    C_batteryf_row = []
    C_PVf_row = []
    LLP_calculated_row = []

    for n in np.arange(C_batterymin, C_batterymax + 1, 500):  # capacity of battery
        SOCmax = n  # Battery Capacity (Wh)
        SOCmin = SOCmax * (1 - DOD)
        w = 0

        # Initialize arrays for this iteration
        SOC = np.zeros(len(I_L))
        I_Load = np.zeros(len(I_L))
        I_Charge = np.zeros(len(I_L))
        I_Discharge = np.zeros(len(I_L))
        I_Deficit = np.zeros(len(I_L))
        I_Damp = np.zeros(len(I_L))
        I_Battery = np.zeros(len(I_L))
        I_net = np.zeros(len(I_L))

        for k in range(len(I_L)):  # Number of hours
            I_net[k] = I_PV[k] - I_L[k]

            # (Routine 5): In Case of the battery is not empty
            if SOCmax > 0:
                # (Routine 5.1): In Case of I_PV=I_Load
                if I_net[k] == 0:
                    I_Load[k] = I_PV[k]
                    I_Damp[k] = 0
                    I_Charge[k] = 0
                    I_Deficit[k] = 0
                    I_Discharge[k] = 0
                    I_Battery[k] = 0
                    if k == 0:
                        SOC[k] = SOC1
                    elif w == 0:  # In case of battery is not discharged on the previous stage
                        SOC[k] = SOC1
                    elif w == 1:  # In case of battery is discharged on the previous stage
                        SOC[k] = SOC[k-1]

                # (Routine 5.2): In Case of I_PV>I_Load
                elif I_net[k] > 0:
                    I_Load[k] = I_L[k]
                    if k == 0:
                        SOC[k] = SOC1
                        I_Damp[k] = I_net[k]
                        I_Charge[k] = 0
                        I_Deficit[k] = 0
                        I_Discharge[k] = 0
                        I_Battery[k] = 0
                    elif k > 0:
                        if w == 0:
                            SOC[k] = SOC1
                            I_Damp[k] = I_net[k]
                        elif w == 1:
                            if SOC[k-1] >= SOC1:
                                SOC[k] = SOC[k-1]
                                I_Damp[k] = I_net[k]
                                I_Charge[k] = 0
                                I_Deficit[k] = 0
                                I_Discharge[k] = 0
                                I_Battery[k] = 0
                            elif SOC[k-1] < SOC1:
                                I_Damp[k] = 0
                                I_Charge[k] = I_net[k]
                                I_Deficit[k] = 0
                                I_Discharge[k] = 0
                                I_Battery[k] = 0
                                # Charging mode
                                B = SOC2
                                V1 = (2 + (0.148 * B)) * ns
                                R1 = float(((0.758 + (0.1309 / (1.06 - B))) * ns) / SOCmax)
                                v = sp.Symbol('v')
                                ee = float(sp.integrate(((K * V1 * I_net[k]) - (D * SOC2 * SOCmax)), (v, 0, t)))
                                SOCx = SOC1 + (SOCmax**-1) * ee
                                SOC2 = SOCx
                                SOC2 = float(SOCx)
                                SOC[k] = SOC[k-1] + abs(SOC1 - SOC2)

                # (Routine 5.3): In Case of I_PV<I_Load
                elif I_net[k] < 0:
                    if w == 0:
                        I_Damp[k] = 0
                        I_Charge[k] = 0
                        I_Deficit[k] = 0
                        I_Discharge[k] = I_L[k] - I_PV[k]
                        I_Battery[k] = I_Discharge[k]
                        I_Load[k] = I_PV[k] + I_Battery[k]
                        # Discharging mode
                        B = SOC2
                        V1 = (1.926 + 0.124 * B) * ns
                        R1 = (0.19 + (0.1037 / (B - 0.14))) * (ns / SOCmax)
                        v = sp.Symbol('v')
                        ee = float(sp.integrate(((K * V1 * I_net[k]) - (D * SOC2 * SOCmax)), (v, 0, t)))
                        SOCx = SOC1 + (SOCmax**-1) * ee
                        SOC2 = SOCx
                        SOC2 = float(SOCx)
                        SOC[k] = SOC2
                        w = w + 1
                    elif w == 1:
                        if SOC[k-1] > SOC3:
                            I_Damp[k] = 0
                            I_Charge[k] = 0
                            I_Deficit[k] = 0
                            I_Discharge[k] = I_L[k] - I_PV[k]
                            I_Battery[k] = I_Battery[k]
                            I_Load[k] = I_L[k]
                            B = SOC2
                            V1 = (1.926 + 0.124 * B) * ns
                            R1 = (0.19 + (0.1037 / (B - 0.14))) * (ns / SOCmax)
                            v = sp.Symbol('v')
                            ee = float(sp.integrate(((K * V1 * I_net[k]) - (D * SOC2 * SOCmax)), (v, 0, t)))
                            SOCx = SOC1 + (SOCmax**-1) * ee
                            SOC2 = SOCx
                            SOC2 = float(SOCx)
                            SOC[k] = SOC[k-1] - abs(SOC1 - SOC2)
                        elif SOC[k-1] <= SOC3:
                            SOC2 = SOC3
                            SOC[k] = SOC3
                            I_Damp[k] = 0
                            I_Charge[k] = 0
                            I_Deficit[k] = I_net[k]
                            I_Discharge[k] = 0
                            I_Battery[k] = 0
                            I_Load[k] = 0

            # In Case of the battery is empty
            else:
                # In Case of I_PV=I_Load
                if I_net[k] == 0:
                    SOC[k] = SOC1
                    I_Load[k] = I_PV[k]
                    I_Damp[k] = 0
                    I_Charge[k] = 0
                    I_Deficit[k] = 0
                    I_Discharge[k] = 0
                    I_Battery[k] = 0
                # In Case of I_PV>I_Load
                elif I_net[k] > 0:
                    SOC[k] = SOC1
                    I_Load[k] = I_L[k]
                    I_Damp[k] = I_net[k]
                    I_Charge[k] = 0
                    I_Deficit[k] = 0
                    I_Discharge[k] = 0
                    I_Battery[k] = 0
                # (Routine 6.3): In Case of I_PV<I_Load
                elif I_net[k] < 0:
                    SOC[k] = SOC1
                    I_Damp[k] = 0
                    I_Charge[k] = 0
                    I_Deficit[k] = I_L[k] - I_PV[k]
                    I_Discharge[k] = 0
                    I_Battery[k] = 0
                    I_Load[k] = I_PV[k]

        E_Excess = np.sum(I_Damp * 230)
        SOC_per = SOC / SOCmax

        C_batteryf_row.append(n)  # Battery Capacity for each hour and PV module
        C_PVf_row.append(m)       # Number of PV modules for each hour
        LLP_calculated_row.append(abs((np.sum(I_Deficit)) / (np.sum(I_L))))
        y = y + 1

    C_batteryf.append(C_batteryf_row)
    C_PVf.append(C_PVf_row)
    LLP_calculated.append(LLP_calculated_row)
    x = x + 1

# Convert to numpy arrays
C_batteryf = np.array(C_batteryf)
C_PVf = np.array(C_PVf)
LLP_calculated = np.array(LLP_calculated)

aa = LLP_calculated.shape
xx = aa[0]
yy = aa[1]
cc = 0

# Initialize arrays to avoid undefined variable error
LLP_ff = []
C_PV_ff = []
C_battery_ff = []

for ii in range(xx):
    for jj in range(yy):
        if LLP_calculated[ii, jj] >= 0.0095 and LLP_calculated[ii, jj] <= 0.0105:
            LLP_ff.append(LLP_calculated[ii, jj])
            C_PV_ff.append(C_PVf[ii, jj])
            C_battery_ff.append(C_batteryf[ii, jj])
            cc = cc + 1

# Check if any valid combinations were found
if len(C_PV_ff) == 0:
    print('No combinations found with LLP between 0.0095 and 0.0105')
    print(f'Minimum LLP found: {np.min(LLP_calculated)}')
    print(f'Maximum LLP found: {np.max(LLP_calculated)}')

    # Find the combination with LLP closest to 0.01
    idx = np.argmin(np.abs(LLP_calculated - 0.01))
    row, col = np.unravel_index(idx, LLP_calculated.shape)

    LLP_ff = [LLP_calculated[row, col]]
    C_PV_ff = [C_PVf[row, col]]
    C_battery_ff = [C_batteryf[row, col]]

    print(f'Using closest LLP value: {LLP_ff[0]}')

# Convert to numpy arrays
LLP_ff = np.array(LLP_ff)
C_PV_ff = np.array(C_PV_ff)
C_battery_ff = np.array(C_battery_ff)

## Initialization of the cost Function
# PV
CC_PV = 456    # Capital Cost for one PV
MC_PV = 6.5    # Maintenance cost of one PV module per year
Ls = 25        # The duration of operation of the system in years
L_PV = 25      # The total lifetime period for PV array

# Battery
Ca_battery = 1200  # Capacity for one battery (Wh)
CC_batwh = 4.8     # Capital Cost for Wh
CC_bat = CC_batwh * Ca_battery  # Capital Cost for one battery
MC_bat = 3.4       # Maintenance cost of one battery per year
L_bat = 5
Y_bat = (Ls / L_bat) - 1  # The expected numbers of the storage battery replacement during the system lifetime
B_rep = 50         # Replacement cost for one battery
CC_B_rep = Y_bat * B_rep  # Cost of the storage battery replacement during the system lifetime

# Charge Controller
CC_cc = 400   # Capital Cost for one Charge Controller
MC_cc = 0     # Maintenance cost of one Charge Controller per year
L_cc = 25     # The total lifetime period for Charge Controller
Y_cc = (Ls / L_cc) - 1  # The expected numbers of the Charge Controller replacement during the system lifetime
N_cc = 1      # Number of Charge Controllers during the system lifetime

# Inverter
CC_inv = 800  # Capital Cost for one Inverter
MC_inv = 0    # Maintenance cost of one Inverter per year
L_inv = 25    # The total lifetime period for Inverter
Y_inv = (Ls / L_inv) - 1  # The expected numbers of the Inverter replacement during the system lifetime
N_inv = 1     # Number of inverters during the system lifetime

# Other Costs
# Circuit Breaker
N_CB = 4      # Number of Circuit breaker
C_CB = 25     # Cost for one circuit breaker
CC_CB = N_CB * C_CB  # Cost for all circuit breakers

# Support Structure
CC_SS = 200

# Civil work
CC_CW = 400

# Total cost for the Other Costs
CC_OC = CC_CB + CC_SS + CC_CW + CC_B_rep
ir = 0.035    # Real Interest Rate
fr = 0.015    # Inflation Rate
ndr = ((1 + ir) / (1 + fr)) - 1  # Net of discount-inflation rate

# Total life cycle cost calculation
LCCx = np.zeros(len(C_PV_ff))
CC_D = np.zeros(len(C_PV_ff))
LCC = np.zeros(len(C_PV_ff))

for kk in range(len(C_PV_ff)):
    LCCx[kk] = (CC_OC / ((((1 + ndr)**Ls) - 1) / ((ndr * ((1 + ndr)**Ls))))) + ((C_PV_ff[kk] * (CC_PV + Ls * MC_PV)) / L_PV) + (((np.ceil(C_battery_ff[kk] / Ca_battery) * CC_bat * (1 + Y_bat)) + (MC_bat * (Ls - Y_bat))) / L_bat) + (((N_cc * CC_cc * (1 + Y_cc)) + (MC_cc * (Ls - Y_cc))) / L_cc) + (((N_inv * CC_inv * (1 + Y_inv)) + (MC_inv * (Ls - Y_inv))) / L_inv)
    CC_D[kk] = (C_PV_ff[kk] * (CC_PV)) + ((np.ceil(C_battery_ff[kk] / Ca_battery) * CC_bat * (1 + Y_bat))) + (N_cc * CC_cc * (1 + Y_cc)) + (N_inv * CC_inv * (1 + Y_inv)) + CC_CB + CC_SS
    LCC[kk] = LCCx[kk] - (((0.13 * (CC_D[kk]))) / ((((1 + ndr)**Ls) - 1) / ((ndr * ((1 + ndr)**Ls)))))

# The optimum sizes of PV and battery combination
MM = np.min(LCC)
II = np.argmin(LCC)
C_PV_best = C_PV_ff[II]
C_battery_best = C_battery_ff[II]

print(f'Best PV Modules Number is: {int(C_PV_best)}')
print(f'Best Battery Capacity is: {int(C_battery_best)}')


Error: Invalid PSH calculation
PSH = nan
S stats: min=nan, max=nan, mean=nan
E_L = 514332900
PSH = 5
N_PV = 1000936.0
N_PVmin = 333646, N_PVmax = 5004680
C_battery = 1607290312.4999998
C_batterymin = 535763437.49999994, C_batterymax = 8036451562.499999


KeyboardInterrupt: 